<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Copy_CNN_Rafa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloads e imports

In [ ]:
!pip install tensorflow
!pip install keras

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import time
import sklearn
import numpy as np
import random as rn
import pandas as pd
start_time = time.time()
from numpy import loadtxt
import matplotlib.pyplot as plt
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split


# load das libs
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import schedules 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.backend import dropoutpout
#, Activation
# from keras.layers import Embedding
# from keras.preprocessing import sequence
# from keras.layers import Dense, Dro
# from keras.layers import Conv1D, Flatten, MaxPooling1D
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences

# random state
sd = 5
np.random.seed(sd)
rn.seed(sd)
os.environ['PYTHONHASHSEED']=str(sd)
tf.random.set_seed(sd)

ImportError: ignored

# Carrega base

In [ ]:
# Base1 Com StopWords
PATH_BASE1 = "/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada.csv"

# Base2 Sem StopWords
PATH_BASE2 = "/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada.csv"

# Base1 Com StopWords
PATH_BASE1_JUNTO_COM_DA_LEILA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv"
#MATRIZ_CBOW_300_BASE_1 = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base1JuntaHateENotHateDaLeila_balanceada_CBOW300.CSV", delimiter=',')

# Base2 Sem StopWords
PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv"
#MATRIZ_CBOW_300_BASE_2_LEILA = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_maior4_menor25_limpissima_balanceada_CBOW300.CSV", delimiter=',')

PATH_BASE_2_CLASSIFICADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_classificada_balanceada.csv" #entrada
MATRIZ_CBOW_300_BASE_2 = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_classificada_balanceada_CBOW300.csv", delimiter=',') #saida

In [ ]:
reviews = pd.read_csv(PATH_BASE_2_CLASSIFICADA, index_col=0)
reviews

# Execução modelo

In [ ]:
# pd.set_option('display.max_colwidth',1000)

texto = reviews['text']
texto

In [ ]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

In [ ]:
def pad(x, length=None):
    return pad_sequences(x, maxlen=length, padding='post')

In [ ]:
def preprocess(x):
    preprocess_x, x_tk = tokenize(x)

    preprocess_x = pad(preprocess_x)

    return preprocess_x, x_tk

In [ ]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
print(reviews.head)

text_tokenized, text_tokenizer = tokenize(texto)

test_pad = pad(text_tokenized)

y = reviews['label']

preproc_texto, texto_tokenizer = preprocess(texto)
    
max_text_length = preproc_texto.shape[1]
text_vocab_size = len(texto_tokenizer.word_index)

worddx = texto_tokenizer.word_index

print("Max sentence length:", max_text_length)
print("Vocabulary size:", text_vocab_size)

In [ ]:
# parametros:
tam_vocab = text_vocab_size # vocabulário do embedding
tam_max = max_text_length # 100
batchsize = 64  # 64
embedding_dimen = 300  # 50 ! 300
filtros = 250 #250
kernel_size = 2 # 3 | 2
hidden_dims = 100 # not used
epochs = 50
lr = 0.0006 # 0.00006
drp = 0.4   # coeficiente de dropout

X_train, X_test, y_train, y_test = train_test_split(test_pad, y, test_size=0.3)

In [ ]:
lr_schedule = schedules.ExponentialDecay(
    initial_learning_rate=lr,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = Adam(learning_rate=lr_schedule) # optimizer = SGD(learning_rate=lr_schedule)

start_time2 = time.time()
#wvec = KeyedVectors.load_word2vec_format('/content/cbow_s300.txt')
#wvec = KeyedVectors.load_word2vec_format("cbow_s50.txt")
#wvec = KeyedVectors.load_word2vec_format("skip_s50.txt")
print(); print("Time to load model: %.5s s.\n" % (time.time() - start_time2))


# gerando a matriz do embedding
# embedding_matrix = np.zeros((tam_vocab+1, embedding_dimen))
# for word, i in worddx.items():
#     if i>=tam_vocab:
#         continue
#     try:
#         embedding_vector = wvec[word]
#         embedding_matrix[i] = embedding_vector
#     except KeyError:
#         embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dimen)

# del(wvec)

# defininindo a camada de embedding utilizando a matriz
# embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],
#                             output_dim=embedding_matrix.shape[1],
#                             weights=[embedding_matrix],
#                             input_length=tam_max,
#                             trainable=False)

#del(embedding_matrix)

#    embedding_vec = embeddings_index.get(word)
#    if embedding_vec is not None:
#        embedding_matrix[index] = embedding_vec

model = Sequential()
# primeira camada se trata do embedding. 
#model.add(embedding_layer)
model.add(Embedding(tam_vocab, embedding_dimen, input_length=tam_max)) 


# input = numero de possiveis palavras, dimensão do espaço(complex!), e 137
#model.add(Embedding(tam_vocab, embedding_dimen, input_length=tam_max)) 

# camada convolucional
# filtros referentes ao output dessa camada,
# tamanho nucleo = 3, se tratando apenas de uma dimensão (conv1d)
# padding 'valid' = sem padding
# função de ativação = relu
model.add(Dropout(drp))
model.add(Conv1D(filtros, kernel_size, padding='same', activation='relu', strides=1))
model.add(Conv1D(filtros, kernel_size, padding='same', activation='relu', strides=1))
model.add(Dropout(drp))
model.add(MaxPooling1D())


# camada convolucional                         'valid|same'
model.add(Conv1D(filtros, kernel_size, padding='same', activation='relu', strides=1))
model.add(Conv1D(filtros, kernel_size, padding='same', activation='relu', strides=1))
model.add(Dropout(drp))
model.add(MaxPooling1D())


# nivela o output para utilização na camada densa
model.add(Flatten())
#model.add(Dense(hidden_dims, activation='relu'))

# regularização por dropout
#model.add(Dropout(0.2))
#model.add(Dense(2, activation='sigmoid')) #FUNÇÃO VAGABUNDA
model.add(Dense(2, activation='softmax')) #FUNÇÃO VAGABUNDA

# calculo de loss adotado = binary_crossentropy/categorical_crossentropy
#model.compile(loss='binary_crossentropy', optimizer=Adam(lr), metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=optimizer, metrics=['accuracy']) 
model.summary()

# Fit the model                                                                                               #val split
history = model.fit(X_train, y_train, batch_size=batchsize, epochs=epochs,validation_data=(X_test, y_test), validation_split = 0.3, verbose=2)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
#print("Accuracy: %.2f%%" % (scores[1]*100))

# tempo de execução total
print(); print("Tempo de execução: %.5s segundos.\n" % (time.time() - start_time))

# plot dos gráficos
ntl.plot_accuracy(history)
ntl.plot_loss(history)

In [ ]:
tf.keras.utils.plot_model(model, to_file='/meuarquivo.png', show_shapes=True)